In [1]:
# implementation of GPT from scratch (but charcter level) and train it on tiny shaksphere

# 2017, Attention is all you need 
# GPT -> Generatively Pretrained Transformer 

In [2]:
with open("data/input.txt", 'r', encoding='utf-8')as f:
    text = f.read()

In [3]:
print(f"no of charcters = {len(text)}")
print(f"no of line= {len(text.splitlines())}")


no of charcters = 1115394
no of line= 40000


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# get all characters 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"vocab_size: {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size: 65


In [6]:
# create a mapping from chars to int and vise-versa
# character level tokeniser 
# long sequences 
stoi = {s:i for i,s in enumerate(chars)}
itos = {v:k for k,v in stoi.items()}
# How lambda works--> some_fucntion_object = lambda input:output_(input)
encode = lambda  s:[stoi[c] for c in s] #encoder: take a string, output a list of integer
decode = lambda  l: ''.join([itos[i] for i in l]) # decode: takes a list of indexes and output a string

In [7]:
# test the encoder and decoder 
print(encode("hello world!"))
print(decode([46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]))
print(decode(encode("hello world!"))) # if it prints "hello world!" it works !

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
hello world!
hello world!


In [8]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data.dtype)
print(data[:10])

torch.Size([1115394])
torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [9]:
# train and validation split 
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
# when the input is {context} the target is {target}
block_size = 8 # time dimention
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    context_itos = [itos[i.item()] for i in context]
    target = y[t]
    target_itos = itos[target.item()]
    print(f"when the context is {context} ->target is {target}")
    print(f"when the context is {context_itos} ->target is '{target_itos}'")
    print("---------")

when the context is tensor([18]) ->target is 47
when the context is ['F'] ->target is 'i'
---------
when the context is tensor([18, 47]) ->target is 56
when the context is ['F', 'i'] ->target is 'r'
---------
when the context is tensor([18, 47, 56]) ->target is 57
when the context is ['F', 'i', 'r'] ->target is 's'
---------
when the context is tensor([18, 47, 56, 57]) ->target is 58
when the context is ['F', 'i', 'r', 's'] ->target is 't'
---------
when the context is tensor([18, 47, 56, 57, 58]) ->target is 1
when the context is ['F', 'i', 'r', 's', 't'] ->target is ' '
---------
when the context is tensor([18, 47, 56, 57, 58,  1]) ->target is 15
when the context is ['F', 'i', 'r', 's', 't', ' '] ->target is 'C'
---------
when the context is tensor([18, 47, 56, 57, 58,  1, 15]) ->target is 47
when the context is ['F', 'i', 'r', 's', 't', ' ', 'C'] ->target is 'i'
---------
when the context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) ->target is 58
when the context is ['F', 'i', 'r', 

In [17]:
# Batch dimention
torch.manual_seed(1337)

block_size = 8 # what is the length of each sequence 
batch_size = 4 # how many independent sequences we will process in parallel 

def get_batch(split):
    """
    Get a mini batch from specified data split
    input: "train" or "val"
    output: x, y 
    y is x moved to right by an index 
    x.shape -> [4,8]
    yb.shape -> [4,8]
    """
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data) - block_size-1,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch("train")
print(xb)
print(xb.shape)
print(yb)
print(yb.shape)

print("---------")
# -----
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        context_itos = [itos[i.item()] for i in context]
        target = yb[b,t]
        target_itos = itos[target.item()]
        print(f"printing sequence:{b+1}, example: {t+1}")
        print(f"when the context is {context_itos} ->target is '{target_itos}'")
        print("---------")

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
torch.Size([4, 8])
---------
printing sequence:1, example: 1
when the context is ['o'] ->target is 'u'
---------
printing sequence:1, example: 2
when the context is ['o', 'u'] ->target is ','
---------
printing sequence:1, example: 3
when the context is ['o', 'u', ','] ->target is ' '
---------
printing sequence:1, example: 4
when the context is ['o', 'u', ',', ' '] ->target is 't'
---------
printing sequence:1, example: 5
when the context is ['o', 'u', ',', ' ', 't'] ->target is 'r'
---------
printing sequence:1, example: 6
when the context is ['o', 'u', ',', ' ', 't', 'r'] ->target is 'i'
---------
printing sequence:1, example: 7
whe

### Biagram Language model

In [44]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

# nn.Module -> Base class for all neural network modules.

class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits of the next token from a look up table
        # nn.Embedding -> A simple lookup table that stores embeddings of a fixed dictionary and size.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx,targets=None):
        # idx and targets are of shape (B,T)
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            # pytorch cross entropy expects second dimention as the channel
            logits = logits.view(B*T , C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is B,T array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions 
            logits, loss = self(idx) # (B, T, C) # self here runs the def forward 
            # get predictions for the last time step 
            logits = logits[:, -1, :] #(B,C) pluck the last time step
            # apply soft max to get the probabilities 
            prob = F.softmax(logits, dim = -1) #(B, C)
            # sample from the distibution 
            idx_next = torch.multinomial(prob, num_samples=1) # (B, 1)
            # append the sampled index into the running sequence
            idx = torch.cat((idx, idx_next), dim= 1) # (B,T+1)
        return idx 
m = BiagramLanguageModel(vocab_size)
logits,loss = m (xb, yb)

print(logits.shape)
print(loss.item())

# generate and decode 
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 100)[0].tolist()))

torch.Size([32, 65])
4.894842624664307

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [49]:
# pytorch optimization object 
# a good learnign rate is 1e-4, but for small networks we can use 1e-3
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [53]:
# training loop 
batch_size = 32 
for _ in range(10000):
    # sample from the batch
    xb, yb = get_batch("train")

    # forward pass and evaluate the loss 
    logits, loss = m(xb,yb)

    # backward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    
    # update 
    optimizer.step()

print(loss.item())

2.464529514312744


In [54]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 100)[0].tolist()))


Tond cks,
Themevelviesthe'; t:
YCURYORKI os it a frimowhion'edeteistink cey weron le'theayolllefovil


In [ ]:
# this was a very simple model, 
# even though we are feeding in the context we only look at the last character 
# the tokens are not talking to each other 

### Maths of self attention

In [125]:
torch.manual_seed(1337)
B, T, C = 6,4, 2 # batch, time and channel 
x = torch.randn(B,T,C)
xbow = torch.zeros(B,T,C)

In [ ]:
# averaging example 
# version. 1 
for b in range(B):
    for t in range(T):
        x_prev = x[b,:t+1] #T, C 
        xbow[b,t] = torch.mean(x_prev, 0)

In [ ]:
# A, B , C example 
A = torch.tril(torch.ones(3,3))
A = A / A.sum(1, keepdim=True)
B = torch.randint(1,10, (3,2)).float()
C = A@B


In [129]:
# apply matrix multiplication to averaging example 
# version 2 
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True) #(4,4)
xbow2 = wei @ x # (T,T) @ (B,T,T) => (B,T,T) @ (B,T,C) => (B,T,C)
torch.allclose(xbow2, xbow)

True

In [ ]:
# version 3 uses softmax 
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim =-1 )
xbow3 = wei @ x
torch.allclose(xbow3, xbow)

True

## Starter code for making Transformer 


In [135]:
""" Script to implement language model"""

# imports
import torch
import torch.nn as nn
import torch.nn.functional as F


# hyperparameters 
batch_size = 32
block_size = 8
n_embd = 32 # no of embedding dimentions
max_iterations = 3000
learning_rate = 1e-2
eval_iters = 200
eval_interval = 300 
device = "cuda" if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1337)
# -------------------------------------

# read the data 
with open("/Users/rahulkrish/Desktop/myrepos/deeplearning/data/input.txt", "r", encoding= 'utf-8')as f:
    text = f.read()
print(f"No of charcters = {len(text)}")
print(f"No of line = {len(text.splitlines())}")

# identify vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"unique character: {''.join(chars)}")
print(f"vocab_size: {vocab_size}")
# create a mapping from chars to int and vise-versa
stoi = {s:i for i,s in enumerate(chars)}
itos = {v:k for k,v in stoi.items()}

# encoder and decoder 
encode = lambda  s:[stoi[c] for c in s] #encoder: take a string, output a list of integer
decode = lambda  l: ''.join([itos[i] for i in l]) # decode: takes a list of indexes and output a string


# train and val split 
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(f"Length of train data: {len(train_data)}")
print(f"Length of validation data: {len(val_data)}")


# get batch 
def get_batch(split):
    """
    Get a mini batch from specified data split
    input: "train" or "val"
    output: x, y 
    y is x moved to right by an index 
    x.shape -> [4,8]
    y.shape -> [4,8]
    """
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data) - block_size-1,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) # move batch to device
    return x,y

# biagram language model 
class TransformerLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # nn.Embedding -> A simple lookup table that stores embeddings of a fixed dictionary and size.
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # embed the identity of the token 
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx,targets=None):
        B, T = idx.shape
        # idx and targets are of shape (B,T)
        token_embd = self.token_embedding_table(idx) # (B,T, n_embd)
        position_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, n_embd)
        x = token_embd + position_embd # (B,T,n_emd)
        logits = self.lm_head(x) # B,T, vocab_size
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            # pytorch cross entropy expects second dimention as the channel
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is B,T array of indices in the current context
        for _ in range(max_new_tokens):
            # crop the context before feeding into self 
            idx_cond = idx[:, -block_size:]
            # get the predictions 
            logits, loss = self(idx_cond) # (B, T, C) # self here runs the def forward 
            # get predictions for the last time step 
            logits = logits[:, -1, :] #(B,C) pluck the last time step
            # apply soft max to get the probabilities 
            prob = F.softmax(logits, dim = -1) #(B, C)
            # sample from the distibution 
            idx_next = torch.multinomial(prob, num_samples=1) # (B, 1)
            # append the sampled index into the running sequence
            idx = torch.cat((idx, idx_next), dim= 1) # (B,T+1)
        return idx
# define model 
model = TransformerLanguageModel()
model = model.to(device)
print("model created successfully!")

# create optimizer 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Evaluate loss 
@torch.no_grad()
def evaluate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# training loop 
print("training started...")
for iter in range(max_iterations):
    #once every once a while evalauate loss on the train adn eval set
    if iter % eval_interval == 0:
        losses = evaluate_loss()
        print(f"step {iter}/ {max_iterations},  train_loss: {losses['train']:.4f}, val_loss: {losses['val']:.4f}")
    # sample from the batch
    xb, yb = get_batch("train")
    # forward pass and evaluate the loss 
    logits, loss = model(xb,yb)
    # backward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # update 
    optimizer.step()
print("training complete!")

# generate sample text 

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens= 500)[0].tolist()))

No of charcters = 1115394
No of line = 40000
unique character: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size: 65
Length of train data: 1003854
Length of validation data: 111540
model created successfully!
training started...
step 0/ 3000,  train_loss: 4.4788, val_loss: 4.4784
step 300/ 3000,  train_loss: 2.5474, val_loss: 2.5587
step 600/ 3000,  train_loss: 2.5103, val_loss: 2.5461
step 900/ 3000,  train_loss: 2.4930, val_loss: 2.5300
step 1200/ 3000,  train_loss: 2.4883, val_loss: 2.5196
step 1500/ 3000,  train_loss: 2.4944, val_loss: 2.5138
step 1800/ 3000,  train_loss: 2.5044, val_loss: 2.5218
step 2100/ 3000,  train_loss: 2.5001, val_loss: 2.5068
step 2400/ 3000,  train_loss: 2.4977, val_loss: 2.5331
step 2700/ 3000,  train_loss: 2.4962, val_loss: 2.4944
training complete!


Stoulowerefrthicaprorheellyouesin.

VERangrey pe My f callereed jeay

Th kiss fouthe t-

s t

RIAndithy thtand pursthare ce gres ughachicalicef ',
The, thean h ure ndrdyousolfive 

In [ ]:
# version 4 self attention 
torch.manual_seed(1337)
B, T, C = 4,8, 32 
x = torch.randn((B,T,C))

# single head of self attention
head_size = 16 
key = nn.Linear(C, head_size) # Linear projection of channel to head_size (independent of query, value)
query = nn.Linear(C, head_size) # Linear projection of channel to head_size (independent of key, value)
value = nn.Linear(C, head_size) # Linear projection of channel to head_size (independent of key, query)

k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

#wei = torch.zeros((T, T))
wei = q @ k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, T) --> (B,T,T)
# wei -> torch.Size([4, 8, 8])
# wei[0] now is: in a batch 1: we have a (T,T) matrix of affinities. 
# how have each token interacted with rest of the tokens 

# now we mask affinities from future
tril = torch.tril(torch.ones(T,T)) #(T,T)
wei = wei.masked_fill(tril == 0, float('-inf')) #(B,T,T)
# normalise them to get probability distributions
wei = F.softmax(wei, dim =-1 ) #(B,T,T)

v = value(x)
out = wei @ v # (B,T,T) @ (B,T,head_size) ==> (B, T, head_size)

In [162]:
# class implementation for self attention head 

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)) )

    def forward(self,x):
        B, T, C = x.shape
        # k,q,v
        k = self.key(x) #(B, T, head_size)
        q = self.query(x) #(B, T, head_size)
        # compute attention scores ('affinities')
        wei = q @ k.transpose(-2, -1)* head_size**-0.5 # (B, T, head_size) @ (B, head_size, T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B,T,T)
        wei = F.softmax(wei, dim =-1 ) #(B,T,T)
        # perform weighted aggregation of values
        v = value(x) #(B, T, head_size)
        out = wei @ v # (B,T,T) @ (B,T,head_size) ==> (B, T, head_size)
        return out

In [ ]:
# class implemetnation fo multihead attention 

class MultiheadAttenion(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat( [h(x) for h in self.heads], dim = -1) # concatenate in channel dimention

In [ ]:
# class implementation for feed forward network 
class FeedForward(nn.Module):
    def __init__(self, fan_in, fan_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(fan_in, fan_out),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.net(x)